In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Input layer
board_input = keras.Input(shape=(2, 3, 3), name='board')
x = layers.Flatten(data_format='channels_first')(board_input)

# Hidden layers
x = layers.Dense(128, activation='softmax')(x)
x = layers.Dense(128, activation='softmax')(x)

# Output layers
x = layers.Dense(9, activation='softmax')(x)
play_dist = layers.Reshape((1, 3, 3), name='play_dist')(x)

model = keras.Model(
  inputs=[board_input],
  outputs=[play_dist],
)

model.compile(
  optimizer='adam',
  loss='mae',
)

model.summary()

2023-01-19 11:11:05.245299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 board (InputLayer)          [(None, 2, 3, 3)]         0         
                                                                 
 flatten (Flatten)           (None, 18)                0         
                                                                 
 dense (Dense)               (None, 128)               2432      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 9)                 1161      
                                                                 
 play_dist (Reshape)         (None, 1, 3, 3)           0         
                                                                 
Total params: 20,105
Trainable params: 20,105
Non-trainable p

2023-01-19 11:11:08.350413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from keras.models import load_model

model = load_model('models/tic_tac_toe.h5')

In [5]:
from src.tic_tac_toe import TicTacToe
from src.self_play import self_train

self_train(games=10, model=model)

model.save('models/tic_tac_toe.h5')

Wins:  {'X': '40.0%', 'O': '30.0%', 'Draw': '30.0%'}
